# Carga del Dataset

En esta sección, cargamos el conjunto de datos `titanic_train.csv` utilizando la biblioteca `pandas`.

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# Cargar el dataset
df = pd.read_csv('datasets/titanic_train.csv')
# Crear una copia del dataframe original
df_original = df.copy()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Preparación de Datos

En esta sección, realizamos el preprocesamiento de los datos, incluyendo el tratamiento de valores nulos, ingeniería de características, conversión de atributos, escalado, selección de características, aplicación de PCA, y la segunda codificación/escalado con guardado de modelos.

## Tratamiento de Valores Nulos

Rellenamos los valores nulos en las columnas `Embarked` y `Cabin` con el modo y 'Unknown', respectivamente.

In [20]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [22]:
# Manejar valores nulos
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df['Cabin'] = df['Cabin'].fillna('Unknown')

In [23]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

## Ingeniería de Características

Creamos nuevas características: `FamilySize` (suma de `SibSp`, `Parch` y 1) y `Title` (extraído de los nombres, con reemplazos para categorías raras).

In [24]:
# Ingeniería de características
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,2,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,1,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,1,Mr


## Conversión de Atributos (Primera Codificación)

Codificamos todas las columnas categóricas (`Sex`, `Embarked`, `Cabin`, `Title`, `Ticket`) usando `OrdinalEncoder`. Este codificador no se guarda.

In [25]:
# Primera codificación (todas las columnas, no guardado)
categorical_cols_all = ['Sex', 'Embarked', 'Cabin', 'Title', 'Ticket']
encoder_all = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df_all_encoded = df.copy()
df_all_encoded[categorical_cols_all] = encoder_all.fit_transform(df[categorical_cols_all])
df_all_encoded.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Title
0,1,0,3,"Braund, Mr. Owen Harris",1.0,22.0,1,0,523.0,7.2500,147.0,2.0,2,2.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.0,38.0,1,0,596.0,71.2833,81.0,0.0,2,3.0
2,3,1,3,"Heikkinen, Miss. Laina",0.0,26.0,0,0,669.0,7.9250,147.0,2.0,1,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.0,35.0,1,0,49.0,53.1000,55.0,2.0,2,3.0
4,5,0,3,"Allen, Mr. William Henry",1.0,35.0,0,0,472.0,8.0500,147.0,2.0,1,2.0


## Predicción de Valores Faltantes de Age

Predecimos los valores faltantes de `Age` usando un modelo de regresión lineal con las características `Pclass`, `SibSp`, `Parch`, `FamilySize`, y `Title`. El modelo no se guarda.

In [26]:
# Predecir valores faltantes de Age
features_age = ['Pclass', 'SibSp', 'Parch', 'FamilySize', 'Title']
X_age = df_all_encoded[features_age].copy()
y_age = df_all_encoded['Age'].copy()

X_train_age = X_age[~y_age.isna()]
y_train_age = y_age[~y_age.isna()]
X_test_age = X_age[y_age.isna()]

model_age = LinearRegression()
model_age.fit(X_train_age, y_train_age)
predicted_ages = model_age.predict(X_test_age)
df_all_encoded.loc[y_age.isna(), 'Age'] = predicted_ages
df_all_encoded.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
FamilySize     0
Title          0
dtype: int64

## Escalado (Primera Vez)

Escalamos todas las características (excluyendo `Survived`, `PassengerId`, y `Name`) usando `RobustScaler`. Este escalador no se guarda.

In [27]:
# Escalar todas las características con RobustScaler (no guardado)
X_all = df_all_encoded.drop(['Survived', 'PassengerId', 'Name'], axis=1)
y = df['Survived'].copy()
scaler_all = RobustScaler()
X_all_scaled = pd.DataFrame(scaler_all.fit_transform(X_all), columns=X_all.columns)
X_all_scaled.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Title
0,0.0,0.0,-0.409929,1.0,0.0,0.515235,-0.312011,0.0,0.0,1.0,0.0
1,-2.0,-1.0,0.605944,1.0,0.0,0.717452,2.461242,-66.0,-2.0,1.0,1.0
2,0.0,-1.0,-0.155961,0.0,0.0,0.919668,-0.282777,0.0,0.0,0.0,-1.0
3,-2.0,-1.0,0.415468,1.0,0.0,-0.797784,1.673732,-92.0,0.0,1.0,1.0
4,0.0,0.0,0.415468,0.0,0.0,0.373961,-0.277363,0.0,0.0,0.0,0.0


## Selección de Características

Calculamos la importancia de las características usando `DecisionTreeClassifier` y definimos las características finales para el modelo.

In [28]:
# Calcular importancia de características con DecisionTreeClassifier
clfDT = DecisionTreeClassifier(random_state=42)
clfDT.fit(X_all_scaled, y)
feature_importances = pd.DataFrame({'feature': X_all_scaled.columns, 'importance': clfDT.feature_importances_})
sorted_features = feature_importances.sort_values(by='importance', ascending=False)
print("\nImportancia de Características desde DecisionTreeClassifier:")
print(sorted_features)


Importancia de Características desde DecisionTreeClassifier:
       feature  importance
1          Sex    0.296168
5       Ticket    0.186361
6         Fare    0.127170
2          Age    0.125361
7        Cabin    0.076855
0       Pclass    0.075763
3        SibSp    0.040517
10       Title    0.026999
9   FamilySize    0.022296
8     Embarked    0.015229
4        Parch    0.007280


In [29]:
# Definir características finales
features = ['Sex', 'Ticket', 'Age', 'Fare', 'Pclass', 'SibSp', 'Title', 'FamilySize']
X_selected = X_all_scaled[features].copy()
X_selected.head()

,Sex,Ticket,Age,Fare,Pclass,SibSp,Title,FamilySize
0,0.0,0.515235,-0.409929,-0.312011,0.0,1.0,0.0,1.0
1,-1.0,0.717452,0.605944,2.461242,-2.0,1.0,1.0,1.0
2,-1.0,0.919668,-0.155961,-0.282777,0.0,0.0,-1.0,0.0
3,-1.0,-0.797784,0.415468,1.673732,-2.0,1.0,1.0,1.0
4,0.0,0.373961,0.415468,-0.277363,0.0,0.0,0.0,0.0


## Aplicación de PCA

Aplicamos PCA a las características seleccionadas, conservando el 99% de la varianza.

In [30]:
# Aplicar PCA
pca = PCA(n_components=0.99)  # Conservar el 99% de la varianza
X_pca = pca.fit_transform(X_selected)

principalDf = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2','PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8'])
principalDf = pd.concat([principalDf, df[['Survived']]], axis=1)
principalDf.info()
principalDf.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   PC1       891 non-null    float64
 1   PC2       891 non-null    float64
 2   PC3       891 non-null    float64
 3   PC4       891 non-null    float64
 4   PC5       891 non-null    float64
 5   PC6       891 non-null    float64
 6   PC7       891 non-null    float64
 7   PC8       891 non-null    float64
 8   Survived  891 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 62.8 KB


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,Survived
0,-0.931095,0.989600,-0.218983,0.601649,-0.015661,0.393988,0.279503,0.064014,0
1,1.842871,-0.910638,1.124463,0.192927,0.172477,-0.295987,1.225476,0.346508,1
2,-1.448755,-0.092028,-1.126969,0.319870,-0.607736,-0.805354,0.393154,0.356075,1
3,1.160311,-0.601569,1.329896,-0.760219,0.882501,-0.040879,0.305333,0.502754,1
4,-1.532810,-0.409078,-0.071810,0.629342,-0.348839,0.107636,-0.122575,-0.019531,0


## Segunda Codificación y Escalado (Guardado)

Realizamos una segunda codificación de las columnas categóricas relevantes (`Sex`, `Title`, `Ticket`) y escalamos las características finales. Guardamos el codificador y el escalador.

In [31]:
# Segunda codificación y escalado (solo características finales, guardado)
# Codificar solo las columnas categóricas relevantes
categorical_cols_final = ['Sex', 'Title', 'Ticket']
encoder_final = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df_final = df.copy()
df_final[categorical_cols_final] = encoder_final.fit_transform(df[categorical_cols_final])

# Actualizar Age en df_final con los valores predichos
df_final.loc[y_age.isna(), 'Age'] = predicted_ages

# Guardar el codificador final
joblib.dump(encoder_final, 'ordinal_encoder.pkl')
print("Codificador ordinal final guardado como ordinal_encoder.pkl")

# Preparar características finales para escalado
X_final = df_final[features].copy()

# Escalar características finales con RobustScaler
scaler_final = RobustScaler()
X_final_scaled = pd.DataFrame(scaler_final.fit_transform(X_final), columns=X_final.columns)

# Guardar el escalador final
joblib.dump(scaler_final, 'robust_scaler.pkl')
print("Escalador robusto final guardado como robust_scaler.pkl")

Codificador ordinal final guardado como ordinal_encoder.pkl
Escalador robusto final guardado como robust_scaler.pkl


## Guardado del Modelo PCA

Guardamos el modelo PCA después de la segunda codificación y escalado.

In [32]:
# Guardar el modelo PCA
joblib.dump(pca, 'pca_model.pkl')
print("Modelo PCA guardado como pca_model.pkl")

Modelo PCA guardado como pca_model.pkl


# Modelado y Evaluación de Modelos

En esta sección, entrenamos y evaluamos el modelo `RandomForestClassifier`. Primero realizamos una validación cruzada con K-Fold, y luego entrenamos y evaluamos el modelo usando `train_test_split`, guardando el modelo final.

## Validación Cruzada con K-Fold

Evaluamos el modelo `RandomForestClassifier` usando validación cruzada con `StratifiedKFold` para obtener una estimación robusta del rendimiento.

In [33]:
# Evaluar el modelo usando validación cruzada
best_model = RandomForestClassifier(
    class_weight='balanced',
    random_state=42,
    max_depth=14,
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_model, X_pca, y, cv=cv, scoring='accuracy')

print("\nResultados de Validación Cruzada:")
print(f"Precisión Media: {np.mean(cv_scores):.4f}")
print(f"Desviación Estándar: {np.std(cv_scores):.4f}")
print(f"Puntuaciones: {cv_scores}")


Resultados de Validación Cruzada:
Precisión Media: 0.8406
Desviación Estándar: 0.0162
Puntuaciones: [0.84357542 0.83707865 0.81460674 0.84269663 0.86516854]


## Entrenamiento y Evaluación con train_test_split

Dividimos los datos en conjuntos de entrenamiento y prueba, entrenamos el modelo en el conjunto de entrenamiento, evaluamos en el conjunto de prueba, y guardamos el modelo.

In [34]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42, stratify=y)

# Entrenar el modelo en el conjunto de entrenamiento
best_model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("\nPrecisión en el conjunto de prueba:", test_accuracy)

# Guardar el modelo RandomForest
joblib.dump(best_model, 'random_forest_model.pkl')
print("Modelo RandomForest guardado como random_forest_model.pkl")

# Imprimir resultados finales
print("\nResultados del Modelo RandomForest:")
print(f"Precisión Media (Validación Cruzada): {np.mean(cv_scores):.4f}")
print(f"Desviación Estándar (Validación Cruzada): {np.std(cv_scores):.4f}")
print(f"Puntuaciones de Validación Cruzada: {cv_scores}")


Precisión en el conjunto de prueba: 0.8603351955307262
Modelo RandomForest guardado como random_forest_model.pkl

Resultados del Modelo RandomForest:
Precisión Media (Validación Cruzada): 0.8406
Desviación Estándar (Validación Cruzada): 0.0162
Puntuaciones de Validación Cruzada: [0.84357542 0.83707865 0.81460674 0.84269663 0.86516854]


In [35]:
# Nuevo train-test split con datos originales (excluyendo Embarked y Cabin)
features_original = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Title', 'FamilySize']
df_original['Title'] = df_original['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df_original['Title'] = df_original['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df_original['Title'] = df_original['Title'].replace(['Mlle', 'Ms'], 'Miss')
df_original['Title'] = df_original['Title'].replace('Mme', 'Mrs')
df_original['FamilySize'] = df_original['SibSp'] + df_original['Parch'] + 1
df_original.loc[y_age.isna(), 'Age'] = predicted_ages

# Codificar variables categóricas en df_original
categorical_cols_orig = ['Sex', 'Title', 'Ticket']
df_original[categorical_cols_orig] = encoder_final.transform(df_original[categorical_cols_orig])

# Escalar características
X_original = df_final[features].copy()  # Use same features as scaler_final was fitted with
X_original_scaled = pd.DataFrame(scaler_final.transform(X_original), columns=X_original.columns)

# Aplicar PCA
X_original_pca = pca.transform(X_original_scaled)

# Nuevo train-test split
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(
    X_original_pca, 
    df_original['Survived'], 
    test_size=0.2, 
    random_state=42, 
    stratify=df_original['Survived']
)

# Predecir con el modelo entrenado
y_pred_orig = best_model.predict(X_test_orig)

# Crear DataFrame con valores originales y predicciones
test_indices = y_test_orig.index
test_results = df_original.loc[test_indices, features_original].copy()
test_results['Survived_Original'] = y_test_orig.values
test_results['Survived_Predicted'] = y_pred_orig

# Guardar resultados
test_results.to_csv('test_predictions.csv', index=False)
print("\nResultados de predicción guardados en 'test_predictions.csv'")
print("\nPrimeras filas de los resultados de prueba:")
print(test_results.head())


Resultados de predicción guardados en 'test_predictions.csv'

Primeras filas de los resultados de prueba:
     Pclass                               Name  Sex        Age  SibSp  Parch  \
565       3               Davies, Mr. Alfred J  1.0  24.000000      2      0   
160       3           Cribb, Mr. John Hatfield  1.0  44.000000      0      1   
553       3  Leeni, Mr. Fahim ("Philip Zenni")  1.0  22.000000      0      0   
860       3            Hansen, Mr. Claus Peter  1.0  41.000000      2      0   
241       3     Murphy, Miss. Katherine "Kate"  0.0  18.423081      1      0   

     Ticket     Fare  Title  FamilySize  Survived_Original  Survived_Predicted  
565   519.0  24.1500    2.0           3                  0                   0  
160   470.0  16.1000    2.0           2                  0                   0  
553   171.0   7.2250    2.0           1                  1                   0  
860   399.0  14.1083    2.0           3                  0                   0  
241   4

In [36]:

# Crear DataFrame con un solo pasajero
data = {
    'PassengerId': [1],
    'Survived': [0],  # Valor real, usado solo para comparar después
    'Pclass': [3],
    'Name': ["Braund, Mr. Owen Harris"],
    'Sex': ["male"],
    'Age': [22],
    'SibSp': [1],
    'Parch': [0],
    'Ticket': ["A/5 21171"],
    'Fare': [7.25],
    'Cabin': [None],
    'Embarked': ['S']
}
df_test_passenger = pd.DataFrame(data)
# Extraer 'Title'
df_test_passenger['Title'] = df_test_passenger['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df_test_passenger['Title'] = df_test_passenger['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir',
                                                                 'Jonkheer', 'Dona'], 'Rare')
df_test_passenger['Title'] = df_test_passenger['Title'].replace(['Mlle', 'Ms'], 'Miss')
df_test_passenger['Title'] = df_test_passenger['Title'].replace('Mme', 'Mrs')

# Crear FamilySize
df_test_passenger['FamilySize'] = df_test_passenger['SibSp'] + df_test_passenger['Parch'] + 1

# Codificar columnas categóricas
categorical_cols_orig = ['Sex', 'Title', 'Ticket']
df_test_passenger[categorical_cols_orig] = encoder_final.transform(df_test_passenger[categorical_cols_orig])

# Seleccionar y escalar features
X_test_single = df_test_passenger[features].copy()
X_test_single_scaled = pd.DataFrame(scaler_final.transform(X_test_single), columns=X_test_single.columns)

# Aplicar PCA
X_test_single_pca = pca.transform(X_test_single_scaled)
# Predecir
y_pred_single = best_model.predict(X_test_single_pca)

# Mostrar resultado
print(f"Resultado real: {df_test_passenger['Survived'].values[0]}")
print(f"Resultado predicho: {y_pred_single[0]}")


Resultado real: 0
Resultado predicho: 0
